<td>
   <a target="_blank" href="https://beewant.com" ><img src="https://media.licdn.com/dms/image/C4E0BAQFKjwZohF7zRA/company-logo_200_200/0/1655303818792?e=2147483647&v=beta&t=fmLrigj1HhqskB4jNk6aDb6XASTUx1SnRvMXWBPtuCY"></a>
</td>
<p><B>Welcome to Beewant Platform Tutorial</B></p>

<td>
    <a href="https://www.linkedin.com/in/beewant/" target="_blank">
        <img src="https://img.shields.io/badge/LinkedIn-0077B5?logo=linkedin&logoColor=white" alt="LinkedIn">
    </a>
</td>

<td>
    <a href="https://beewant.slack.com/" target="_blank">
        <img src="https://img.shields.io/badge/Slack-4A154B?logo=slack&logoColor=white" alt="Slack">
    </a>
</td>

Learn how to send annotations for caption generation tasks using the Beewant platform's generative templates. This tutorial provides a comprehensive guide on annotating caption generation tasks, enabling you to input detailed descriptions for images. Explore the process of sending annotations for caption generation tasks, ensuring accurate and contextually relevant captions for images. Follow step-by-step instructions to enhance the annotation process for caption generation tasks on the Beewant platform.

In [ ]:
import json
import requests
import concurrent.futures

# API Key and Client
Provide a valid api key below in order to properly connect to the Beewant Client.

In [ ]:
# To get your API key go to: Workspace settings -> Account -> Access Token
beewant_api_key = ""

In [ ]:
# The URL to make the API request to
url = "https://beewant.com/api/projects"

# Replace {API_KEY} with your actual API key for authentication
headers = {
    'Authorization': f'Token {beewant_api_key}'
}

# Sending a GET request to the API endpoint with the headers for authentication
response = requests.request("GET", url, headers=headers)

# Check the response status code to verify the request was successful
if response.status_code == 200:
    # If the status code is 200, the request was successful, and we can access the JSON data
    data = response.json()
    print("API response:")
    print(data)
else:
    # If the status code is not 200, there was an error in the request
    print(f"Error: {response.status_code} - {response.text}")

## Data Transformation

Before proceeding further, it's essential to ensure that the input data file is properly formatted to align with the requirements of the platform. In this case, the input data file can be in any format, such as JSON, CSV, or other structured data formats. However, it needs to be formatted correctly to match the input format expected by the platform.

For instance, consider the following example of input data in JSON format:

```json
{
  "rows": [
    {
      "row_idx": 1,
      "row": {
        "link": "https://example.com/image1.jpg",
        "caption": "A beautiful sunset over the mountains"
      }
    },
    {
      "row_idx": 2,
      "row": {
        "link": "https://example.com/image2.jpg",
        "caption": "A serene lake surrounded by lush greenery"
      }
    }
  ]
}

In [ ]:
# insert the identifier of the project you want to upload the data to
# the projet must be created in the platform before running this script and its template must be set to "caption generation"
project_id = ''

# read content from json file
with open('example_data.json', 'r') as file:
    input_json = json.load(file)

output_list = []

for row in input_json["rows"]:
    item = {
        "id": row["row_idx"],
        "annotations": [],
        "file_upload": row["row"]["link"].split("/")[-1],
        "drafts": [
            {
                "result": [
                    {
                        "type": "textarea",
                        "value": {
                            "text": [row["row"]["caption"]]
                        },
                        "origin": "manual",
                        "to_name": "image",
                        "from_name": "caption"
                    }
                ],
            }
        ],
        "data": {
            "captioning": row["row"]["link"]
        },
        "project": project_id,
    }
    output_list.append(item)

## Send annotations

This code snippet serves the purpose of interacting with the Beewant platform API to send task and annotation objects. Here's a breakdown of its functionality:

1. **Send Task Object**:
   - It constructs a task object using the data from the input item, which includes necessary information for creating a task in a specified project.
   - The task object is then sent to the task endpoint of the Beewant platform via a POST request.

2. **Handle Task Response**:
   - Upon receiving a response from the platform, it prints the response content and status code to the console for debugging.

3. **Send Annotation Object**:
   - If the task creation is successful (indicated by a status code of 201), it extracts the task ID from the response and constructs an annotation object from the drafts field of the input item.
   - This annotation object is then sent to the annotation endpoint of the Beewant platform via a POST request.

4. **Handle Annotation Response**:
   - Similar to task response handling, it prints the response content and status code after sending the annotation object.

5. **Handle Failure**:
   - If the task creation fails (status code other than 201), it prints a message indicating the failure and skips the annotation creation process.

This code snippet is essential for uploading data and annotations to a specific project on the Beewant platform, facilitating seamless interaction with its API.

In [ ]:
def send_task_annotations(item):
    task_object = {"data": item["data"]}

    # Send task object to task endpoint
    url = f'https://beewant.com/api/projects/{project_id}/tasks/'
    response = requests.post(url, headers={'Authorization': f'Token {beewant_api_key}'}, data=json.dumps(task_object))
    print("Task Response:")
    print(response.content)
    print(response.status_code)

    # Send annotation object to annotation endpoint
    if response.status_code == 201:  # Assuming 201 indicates a successful task creation
        task_id = response.json()["id"]
        annotation_object = item["drafts"][0]
        annotation_url = f'https://beewant.com/api/tasks/{task_id}/annotations/'
        response = requests.post(annotation_url, headers={'Authorization': f'Token {beewant_api_key}'}, data=json.dumps(annotation_object))
        print("Annotation Response:")
        print(response.content)
        print(response.status_code)
    else:
        print("Task creation failed, skipping annotation creation.")

In [ ]:
# Create an executor with a maximum of 5 threads
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Submit each task annotation object to the executor
    futures = [executor.submit(send_task_annotations, item) for item in output_list]

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        future.result()